In [ ]:


# Load environment variables
load_dotenv()



# Configure generative AI and agentops
genai.configure(api_key=OPENAI_API_KEY)
agentops.init(
    api_key=AGENTOPS_API_KEY,
    skip_auto_end_session=True,
)

intro_prompt = (
    "Welcome to PublishMate! I am your research assistant mate here to help you with your academic paper journey.\n"
    "I will guide you step-by-step to find trending topics, recent papers, summaries, "
    "research gaps, and help with paper writing. \nLet's get started!\n"
)

def welcome_message():
    print(intro_prompt)

# Run this at the very beginning
welcome_message()

output_dir = './PublishMate_agent_ouput'
os.makedirs(output_dir, exist_ok=True)

user_input = input("Enter your research field or keyword: ")

# class GeminiWrapper(BaseLLM):
#     def __init__(self, api_key: str, model_name: str = "gemini/gemini-1.5-pro"):
#         if not api_key:
#             raise ValueError("Gemini API key must be provided.")
#         genai.configure(api_key=api_key)
#         self.model_name = model_name
#         self.model = genai.GenerativeModel(model_name)
    
#     def chat(self, messages: list[dict], **kwargs):
#         contents_list = []
#         for msg in messages:
#             text = msg.get("content", "")
#             content_obj = Content(parts=[Part(text=text)])
#             contents_list.append(content_obj)
#         response = self.model.generate_content(
#             contents=contents_list,
#             generation_config=self.config,
#             safety_settings=self.safety_config,
#             stream=self.streaming,
#             tools=[]
#         )
#         return {"content": response.candidates[0].content.parts[0].text}

#     def call(self, prompt: str, **kwargs):
#         try:
#             response = self.model.generate_content(prompt)
#             return getattr(response, "text", str(response))
#         except Exception as e:
#             return f"Error: {e}"

# # Initialize GeminiWrapper with your API key
# gemini_llm = GeminiWrapper(api_key=GEMINI_API_KEY)

output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)


# Define your Agent using this LLM
trending_topics_agent = Agent(
    role="Trending Topics Identification Agent",
    goal=f"Identify trending research topics in {user_input} to assist in paper writing.",
    backstory="An AI-powered assistant to help researchers find hot topics.",
    llm=basic_llm,
    verbose=True,
)

# Define your Task
trending_topics_task = Task(
    name="Identify Trending Topics",
    description=f"Generate a list of top 5 trending {user_input} research topics.",
    agent=trending_topics_agent,
    expected_output="list[str]",
)

# Define the Crew
crew_agents = Crew(
    name="PublishMate Crew",
    description="A crew of agents designed to assist with academic research and paper writing.",
    agents=[trending_topics_agent],
    tasks=[trending_topics_task],
)

result = crew_agents.kickoff()
print(result)


Welcome to PublishMate! I am your research assistant mate here to help you with your academic paper journey.
I will guide you step-by-step to find trending topics, recent papers, summaries, research gaps, and help with paper writing. 
Let's get started!

# Agent: Trending Topics Identification Agent
## Task: Generate a list of top 5 trending AI research topics.


# Agent: Trending Topics Identification Agent
## Final Answer: 
['Large Language Models (LLMs) and their applications (e.g., improved efficiency, reasoning capabilities, bias mitigation, and safety)', 'Generative AI and its ethical implications (e.g., copyright, misinformation, and job displacement)', 'AI for Science (e.g., drug discovery, materials science, and climate modeling)', 'Reinforcement Learning and its applications in robotics and autonomous systems', 'Explainable AI (XAI) and trustworthy AI (e.g., model interpretability, fairness, and accountability)']




🖇 AgentOps: Session Replay for default.session trace: https://app.agentops.ai/sessions?trace_id=bdf889373a43c84f8ba85da2af948ecc


['Large Language Models (LLMs) and their applications (e.g., improved efficiency, reasoning capabilities, bias mitigation, and safety)', 'Generative AI and its ethical implications (e.g., copyright, misinformation, and job displacement)', 'AI for Science (e.g., drug discovery, materials science, and climate modeling)', 'Reinforcement Learning and its applications in robotics and autonomous systems', 'Explainable AI (XAI) and trustworthy AI (e.g., model interpretability, fairness, and accountability)']


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict
import os

# Example LLM placeholder, replace with your actual LLM instance
deepseek_llm = "deepseek_llm_instance"

output_dir = "./output"  # adjust as needed

# Common intro prompt for all agents
intro_prompt = (
    "Welcome to PublishMate! I am your research assistant mate here to help you with your academic "
    "paper journey. I will guide you step-by-step to find trending topics, recent papers, summaries, "
    "research gaps, and help with paper writing. Let's get started!"
)

# ========== Agent 1: Trending Topics Agent ==========

class TrendingTopicsOutput(BaseModel):
    topics: List[Dict[str, str]] = Field(..., title="Trending topics with description", min_items=1)

trending_topics_agent = Agent(
    role="Trending Topics Identification Agent",
    goal="\n".join([
        intro_prompt,
        "You are an expert research assistant that identifies the latest trending topics in a given academic field.",
        "Given a user-supplied research field or keyword, generate a detailed list of the top 3-5 trending topics reflecting recent advances and high interest areas."
    ]),
    backstory="Designed to guide users by providing the most relevant and current trending research topics in their specified field.",
    llm=deepseek_llm,
    verbose=True,
)

trending_topics_task = Task(
    description="\n".join([
        "User inputs a research field or keyword (e.g., 'AI agents', 'transformers').",
        "Provide a list of 3 to 5 trending topics with a brief description for each.",
        "Focus on recent research interests supported by publication trends.",
        "Output in JSON format with 'topics' as list of objects {name, description}."
    ]),
    expected_output="JSON object with list of trending topics and descriptions.",
    output_json=TrendingTopicsOutput,
    output_file=os.path.join(output_dir, "step_1_trending_topics.json"),
    agent=trending_topics_agent,
)

# ========== Agent 2: Recent Papers Retrieval Agent ==========

class PaperInfo(BaseModel):
    title: str
    authors: str
    abstract: str
    year: str
    url: str

class RecentPapersOutput(BaseModel):
    topic_papers: Dict[str, List[PaperInfo]] = Field(..., title="Recent papers grouped by topic")

recent_papers_agent = Agent(
    role="Recent Papers Retrieval Agent",
    goal="\n".join([
        intro_prompt,
        "You are a research paper search assistant.",
        "Given a list of trending topics, retrieve 3 recent, relevant publications per topic.",
        "Select papers from reputable sources published within the last 2 years."
    ]),
    backstory="Fetches and organizes recent academic papers for user review.",
    llm=deepseek_llm,
    verbose=True,
)

recent_papers_task = Task(
    description="\n".join([
        "Input is a list of trending topics.",
        "For each topic, find 3 papers with title, authors, abstract, year, and link.",
        "Focus on papers from last 2 years from reputable conferences or journals.",
        "Output JSON grouped by topic."
    ]),
    expected_output="JSON with topics as keys and list of paper info objects as values.",
    output_json=RecentPapersOutput,
    output_file=os.path.join(output_dir, "step_2_recent_papers.json"),
    agent=recent_papers_agent,
)

# ========== Agent 3: Paper Summarization Agent ==========

class PaperSummariesOutput(BaseModel):
    summaries: Dict[str, str] = Field(..., title="Paper title mapped to its summary")

paper_summarization_agent = Agent(
    role="Academic Paper Summarization Agent",
    goal="\n".join([
        intro_prompt,
        "Summarize each research paper into concise 100-150 word summaries.",
        "Focus on main findings, methods, contributions, and note diagrams if mentioned."
    ]),
    backstory="Provides clear and informative summaries to help users understand research papers quickly.",
    llm=deepseek_llm,
    verbose=True,
)

paper_summarization_task = Task(
    description="\n".join([
        "Input is a list of papers with metadata and abstracts.",
        "Produce a short summary for each paper highlighting key points and visuals if any.",
        "Output JSON mapping paper titles to summaries."
    ]),
    expected_output="JSON object mapping paper titles to summaries.",
    output_json=PaperSummariesOutput,
    output_file=os.path.join(output_dir, "step_3_paper_summaries.json"),
    agent=paper_summarization_agent,
)

# ========== Agent 4: Research Gap and Suggestion Agent ==========

class ResearchGapOutput(BaseModel):
    research_gaps: List[str] = Field(..., title="List of research gaps and suggestions")

research_gap_agent = Agent(
    role="Research Gap Identification and Suggestion Agent",
    goal="\n".join([
        intro_prompt,
        "Analyze summaries to identify gaps, limitations, and propose research directions or improvements.",
        "Use a friendly and encouraging tone suitable for beginners."
    ]),
    backstory="Helps users find novel contributions by highlighting unexplored areas and providing ideas.",
    llm=deepseek_llm,
    verbose=True,
)

research_gap_task = Task(
    description="\n".join([
        "Input is paper summaries.",
        "Output a list of research gaps, limitations, and suggestions for future research.",
        "Encourage beginners by providing feasible ideas."
    ]),
    expected_output="JSON list of research gaps and improvement suggestions.",
    output_json=ResearchGapOutput,
    output_file=os.path.join(output_dir, "step_4_research_gaps.json"),
    agent=research_gap_agent,
)

# ========== Agent 5: Paper Structure and Writing Guide Agent ==========

class PaperStructureSection(BaseModel):
    section: str
    tips: str

class PaperStructureOutput(BaseModel):
    paper_structure: List[PaperStructureSection] = Field(..., title="Paper structure sections and writing tips")

paper_structure_agent = Agent(
    role="Paper Structure and Writing Guide Agent",
    goal="\n".join([
        intro_prompt,
        "Provide a clear outline for structuring an academic paper.",
        "Give detailed tips on what to write in each section to help beginners.",
        "Include motivational and supportive writing advice."
    ]),
    backstory="Guides users through the paper writing process with a beginner-friendly approach.",
    llm=deepseek_llm,
    verbose=True,
)

paper_structure_task = Task(
    description="\n".join([
        "Input is the chosen research topic.",
        "Output a recommended paper structure with sections and detailed writing tips for each.",
        "Help beginners understand what content belongs in each part of the paper."
    ]),
    expected_output="JSON list of sections with writing tips.",
    output_json=PaperStructureOutput,
    output_file=os.path.join(output_dir, "step_5_paper_structure.json"),
    agent=paper_structure_agent,
)
